# An example of using the python eTaSL driver in a python notebook

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML


In [ ]:
# change the width of the notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Simulating the eTaSL controller using the high-level eTaSL driver

In [ ]:
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad
e = etasl_simulator(regularization_factor= 0.00001)
e.readTaskSpecificationFile("etasl_robot_ur10.lua")
e.readTaskSpecificationFile("etasl_simple_task.lua")

pos_lbl = ['shoulder_pan_joint','shoulder_lift_joint', 'elbow_joint', 
           'wrist_1_joint', 'wrist_2_joint',  'wrist_3_joint', 'f1']
initial_jpos = np.array([0, -np.pi*0.6 , np.pi*0.6,0,0,0,0])



N=4000
dt=0.005
time = np.arange(0,N)*dt
inp_lbl=['tgt_x','tgt_y','tgt_z']
inp=np.zeros((N, len(inp_lbl)))
inp[:,0] = np.sin(time)*0.15 + 0.7
inp[:,1] = time*0
inp[:,2] = time*0 + 0.3
inpvel = np.zeros((N, len(inp_lbl)))
inpvel[:,0] = np.cos(time)*0.15
inpvel[:,1] = time*0
inpvel[:,2] = time*0

e.setInputTable(inp_lbl,inp,inpvel)
e.initialize(initial_jpos, pos_lbl)
e.simulate(N=N,dt=dt)

# Plotting (using the Bokeh library to provide interactive plots)

Library can also be used to generate an HTML file for the plots

In [ ]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

TOOLTIPS = [
    ("(time,value)", "($x, $y)")
]

p = figure(plot_width=400, plot_height=400, tooltips=TOOLTIPS,
           title="Mouse over the dots")

print("joint positions : ")
print(e.POS_LBL)
p=[]
for i in range(0,e.POS.shape[1]):
    f = figure(x_axis_label='time [s]', 
            y_axis_label= e.POS_LBL[i],     
            tooltips=TOOLTIPS,
            tools = "pan,wheel_zoom,box_zoom,hover,save,reset")
    if i==0:
        fx=f.x_range
    else:
        f.x_range=fx
    f.line(e.TIME, e.POS[:,i], line_width=2)
    p.append(f)

pg = gridplot( [[ p[0], p[1], p[2],p[3] ],[ p[4], p[5], p[6], None]], 
            plot_width=380, plot_height=250,
            tools = "pan,wheel_zoom,box_zoom,hover,save,reset" ) 
show(pg)

print("joint velocities : ")
print(e.POS_LBL)
p=[]
for i in range(0,e.POS.shape[1]):
    f = figure(x_axis_label='time [s]', 
            y_axis_label= "velocity of "+e.POS_LBL[i],    
            tooltips=TOOLTIPS,
            tools = "pan,wheel_zoom,box_zoom,hover,save,reset")
    if i==0:
        fx=f.x_range
    else:
        f.x_range=fx
    f.line(e.TIME, e.VEL[:,i], line_width=2)
    p.append(f)

pg = gridplot( [[ p[0], p[1], p[2],p[3] ],[ p[4], p[5], p[6], None]], 
            plot_width=380, plot_height=250,
            tools = "pan,wheel_zoom,box_zoom,hover,save,reset" ) 
show(pg)

print("error:")

pw=500
ph=400
# output to static HTML file

p1 = figure(x_axis_label='time', 
            y_axis_label= "error x",
            plot_width=pw,
            plot_height=ph)
p1.line(e.TIME, e.OUTP[:,0] - e.INP[:,0], line_width=2)

p2 = figure(x_axis_label='time', 
            y_axis_label= "error y",
            plot_width=pw,
            plot_height=ph,
            x_range = p1.x_range)
p2.line(e.TIME, e.OUTP[:,1] - e.INP[:,1], line_width=2)

p3 = figure(x_axis_label='time', 
            y_axis_label= "error z",
            plot_width=pw,
            plot_height=ph,
            x_range = p1.x_range)
p3.line(e.TIME, e.OUTP[:,2] - e.INP[:,2], line_width=2)

pg = gridplot( [ [p1,p2, p3 ] ])    
show(pg)

# Using matplotlib

In [ ]:
plotw=24
w=4;h=2
plt.figure(figsize=(plotw,9))
for i in range(e.POS.shape[1]):
    ax=plt.subplot(h,w,i+1)
    ax.plot(e.TIME, e.POS[:,i]*180.0/np.pi)
    plt.ylabel(e.POS_LBL[i] +" [deg]")
    plt.xlabel('time [s]')
    
w=4;h=2
plt.figure(figsize=(plotw,9))
for i in range(e.VEL.shape[1]):
    ax=plt.subplot(h,w,i+1)
    ax.plot(e.TIME, e.VEL[:,i])
    plt.ylabel(e.POS_LBL[i] +" [rad/s]")
    plt.xlabel('time [s]')

w=4;h=1;
plt.figure(figsize=(plotw,4))
for i in range(e.INP.shape[1]):
    plt.subplot(h,w,i+1)
    plt.plot(e.TIME, e.INP[:,i])
    plt.ylabel(e.INP_LBL[i])
    plt.xlabel('time [s]')
    
w=4;h=1;
plt.figure(figsize=(plotw,4))
for i in range(e.OUTP.shape[1]):
    plt.subplot(h,w,i+1)
    plt.plot(e.TIME, e.OUTP[:,i])
    plt.ylabel(e.OUTP_LBL[i])
    plt.xlabel('time [s]')    
    
    
plt.figure(figsize=(plotw,4))
plt.subplot(1,3,1)
plt.plot(e.TIME, e.OUTP[:,0] - e.INP[:,0])
plt.xlabel('time [s]')
plt.ylabel('error x')

plt.subplot(1,3,2)
plt.plot(e.TIME, e.OUTP[:,1] - e.INP[:,1])
plt.xlabel('time [s]')
plt.ylabel('error y')

plt.subplot(1,3,3)
plt.plot(e.TIME, e.OUTP[:,2] - e.INP[:,2])
plt.xlabel('time [s]')
plt.ylabel('error z')